In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
try:
    from tensorflow import keras
    from tensorflow.keras import layers
    KERAS_AVAILABLE = True
except:
    KERAS_AVAILABLE = False
    print("Warning: TensorFlow not available. Deep learning models will be skipped.")

# Statistical Models
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# --- CONFIGURATION ---
import os
BASE_DIR = os.path.dirname(os.getcwd()) 
FORECAST_FILE = os.path.join(BASE_DIR, "data", "processed", "forecast_ready.parquet")
OUTPUT_DIR = os.path.join(BASE_DIR, "outputs")
PREDICTIONS_DIR = os.path.join(BASE_DIR, "predictions")
os.makedirs(PREDICTIONS_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Forecast horizon
FORECAST_START = '2026-01-01'
FORECAST_END = '2026-01-31'

# --- 1. LOAD DATA ---
print("="*80)
print("COMPREHENSIVE PM2.5 FORECASTING PIPELINE")
print("9 Models: 3 Statistical | 3 Machine Learning | 3 Deep Learning")
print("="*80)

df = pd.read_parquet(FORECAST_FILE)
df['Date'] = pd.to_datetime(df['Date'])

# --- 2. FEATURE ENGINEERING ---
def create_features(data, city_name):
    """Create time-based and lag features"""
    df_city = data[data['City'] == city_name].copy()
    df_city = df_city.sort_values('Date').reset_index(drop=True)
    
    # Time features
    df_city['year'] = df_city['Date'].dt.year
    df_city['month'] = df_city['Date'].dt.month
    df_city['day'] = df_city['Date'].dt.day
    df_city['dayofweek'] = df_city['Date'].dt.dayofweek
    df_city['dayofyear'] = df_city['Date'].dt.dayofyear
    df_city['quarter'] = df_city['Date'].dt.quarter
    df_city['weekofyear'] = df_city['Date'].dt.isocalendar().week
    
    # Cyclical encoding
    df_city['month_sin'] = np.sin(2 * np.pi * df_city['month'] / 12)
    df_city['month_cos'] = np.cos(2 * np.pi * df_city['month'] / 12)
    df_city['day_sin'] = np.sin(2 * np.pi * df_city['dayofweek'] / 7)
    df_city['day_cos'] = np.cos(2 * np.pi * df_city['dayofweek'] / 7)
    
    # Lag features
    for lag in [1, 2, 3, 7, 14, 30]:
        df_city[f'lag_{lag}'] = df_city['median'].shift(lag)
    
    # Rolling statistics
    for window in [7, 14, 30]:
        df_city[f'rolling_mean_{window}'] = df_city['median'].shift(1).rolling(window).mean()
        df_city[f'rolling_std_{window}'] = df_city['median'].shift(1).rolling(window).std()
        df_city[f'rolling_min_{window}'] = df_city['median'].shift(1).rolling(window).min()
        df_city[f'rolling_max_{window}'] = df_city['median'].shift(1).rolling(window).max()
    
    # EWMA
    df_city['ewm_7'] = df_city['median'].shift(1).ewm(span=7).mean()
    df_city['ewm_30'] = df_city['median'].shift(1).ewm(span=30).mean()
    
    return df_city

# --- 3. TIME SERIES CV ---
def time_series_cv_split(data, n_splits=5, test_size=30):
    """Create time series cross-validation splits"""
    splits = []
    total_size = len(data)
    
    for i in range(n_splits):
        test_end = total_size - i * test_size
        test_start = test_end - test_size
        train_end = test_start
        
        if train_end < 100:
            break
            
        train_idx = data.index[:train_end]
        test_idx = data.index[test_start:test_end]
        splits.append((train_idx, test_idx))
    
    return splits[::-1]

# --- 4. MODEL DEFINITIONS ---

# STATISTICAL MODELS
def train_arima(y_train, y_test):
    """ARIMA - AutoRegressive Integrated Moving Average"""
    try:
        model = ARIMA(y_train, order=(5, 1, 2))
        fitted = model.fit()
        forecast = fitted.forecast(steps=len(y_test))
        return forecast.values
    except:
        return np.full(len(y_test), y_train.mean())

def train_sarima(y_train, y_test):
    """SARIMA - Seasonal ARIMA"""
    try:
        model = SARIMAX(y_train, order=(2, 1, 2), seasonal_order=(1, 1, 1, 7))
        fitted = model.fit(disp=False)
        forecast = fitted.forecast(steps=len(y_test))
        return forecast.values
    except:
        return np.full(len(y_test), y_train.mean())

def train_exponential_smoothing(y_train, y_test):
    """Exponential Smoothing (Holt-Winters)"""
    try:
        model = ExponentialSmoothing(y_train, seasonal_periods=7, trend='add', seasonal='add')
        fitted = model.fit()
        forecast = fitted.forecast(steps=len(y_test))
        return forecast.values
    except:
        return np.full(len(y_test), y_train.mean())

# MACHINE LEARNING MODELS
def train_random_forest(X_train, y_train, X_test):
    """Random Forest Regressor"""
    model = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    return model.predict(X_test), model

def train_gradient_boosting(X_train, y_train, X_test):
    """Gradient Boosting Regressor"""
    model = GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
    model.fit(X_train, y_train)
    return model.predict(X_test), model

def train_svr(X_train, y_train, X_test):
    """Support Vector Regression"""
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    model = SVR(kernel='rbf', C=100, gamma='scale', epsilon=0.1)
    model.fit(X_train_scaled, y_train)
    return model.predict(X_test_scaled), (model, scaler)

# DEEP LEARNING MODELS
def create_sequences(data, lookback=30):
    """Create sequences for deep learning"""
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i-lookback:i])
        y.append(data[i])
    return np.array(X), np.array(y)

def train_lstm(y_train, y_test, lookback=30):
    """LSTM - Long Short-Term Memory Network"""
    if not KERAS_AVAILABLE:
        return np.full(len(y_test), y_train.mean()), None
    
    try:
        # Normalize
        train_mean, train_std = y_train.mean(), y_train.std()
        y_train_norm = (y_train - train_mean) / train_std
        
        # Create sequences
        X_train, y_train_seq = create_sequences(y_train_norm.values, lookback)
        
        if len(X_train) < 50:
            return np.full(len(y_test), y_train.mean()), None
        
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        
        # Build model
        model = keras.Sequential([
            layers.LSTM(50, activation='relu', return_sequences=True, input_shape=(lookback, 1)),
            layers.Dropout(0.2),
            layers.LSTM(50, activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(25, activation='relu'),
            layers.Dense(1)
        ])
        
        model.compile(optimizer='adam', loss='mse')
        model.fit(X_train, y_train_seq, epochs=50, batch_size=32, verbose=0)
        
        # Forecast
        last_sequence = y_train_norm.values[-lookback:]
        predictions = []
        
        for _ in range(len(y_test)):
            X_pred = last_sequence.reshape((1, lookback, 1))
            pred = model.predict(X_pred, verbose=0)[0, 0]
            predictions.append(pred)
            last_sequence = np.append(last_sequence[1:], pred)
        
        # Denormalize
        predictions = np.array(predictions) * train_std + train_mean
        return predictions, model
    except:
        return np.full(len(y_test), y_train.mean()), None

def train_gru(y_train, y_test, lookback=30):
    """GRU - Gated Recurrent Unit"""
    if not KERAS_AVAILABLE:
        return np.full(len(y_test), y_train.mean()), None
    
    try:
        train_mean, train_std = y_train.mean(), y_train.std()
        y_train_norm = (y_train - train_mean) / train_std
        
        X_train, y_train_seq = create_sequences(y_train_norm.values, lookback)
        
        if len(X_train) < 50:
            return np.full(len(y_test), y_train.mean()), None
        
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        
        model = keras.Sequential([
            layers.GRU(50, activation='relu', return_sequences=True, input_shape=(lookback, 1)),
            layers.Dropout(0.2),
            layers.GRU(50, activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(25, activation='relu'),
            layers.Dense(1)
        ])
        
        model.compile(optimizer='adam', loss='mse')
        model.fit(X_train, y_train_seq, epochs=50, batch_size=32, verbose=0)
        
        last_sequence = y_train_norm.values[-lookback:]
        predictions = []
        
        for _ in range(len(y_test)):
            X_pred = last_sequence.reshape((1, lookback, 1))
            pred = model.predict(X_pred, verbose=0)[0, 0]
            predictions.append(pred)
            last_sequence = np.append(last_sequence[1:], pred)
        
        predictions = np.array(predictions) * train_std + train_mean
        return predictions, model
    except:
        return np.full(len(y_test), y_train.mean()), None

def train_cnn_lstm(y_train, y_test, lookback=30):
    """CNN-LSTM Hybrid"""
    if not KERAS_AVAILABLE:
        return np.full(len(y_test), y_train.mean()), None
    
    try:
        train_mean, train_std = y_train.mean(), y_train.std()
        y_train_norm = (y_train - train_mean) / train_std
        
        X_train, y_train_seq = create_sequences(y_train_norm.values, lookback)
        
        if len(X_train) < 50:
            return np.full(len(y_test), y_train.mean()), None
        
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        
        model = keras.Sequential([
            layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(lookback, 1)),
            layers.MaxPooling1D(pool_size=2),
            layers.LSTM(50, activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(25, activation='relu'),
            layers.Dense(1)
        ])
        
        model.compile(optimizer='adam', loss='mse')
        model.fit(X_train, y_train_seq, epochs=50, batch_size=32, verbose=0)
        
        last_sequence = y_train_norm.values[-lookback:]
        predictions = []
        
        for _ in range(len(y_test)):
            X_pred = last_sequence.reshape((1, lookback, 1))
            pred = model.predict(X_pred, verbose=0)[0, 0]
            predictions.append(pred)
            last_sequence = np.append(last_sequence[1:], pred)
        
        predictions = np.array(predictions) * train_std + train_mean
        return predictions, model
    except:
        return np.full(len(y_test), y_train.mean()), None

# --- 5. COMPREHENSIVE MODEL EVALUATION ---
def evaluate_all_models(df_city, city_name):
    """Train and evaluate all 9 models"""
    
    print(f"\n{'='*80}")
    print(f"City: {city_name}")
    print('='*80)
    
    # Prepare data
    feature_cols = [col for col in df_city.columns 
                   if col not in ['Date', 'City', 'median'] and not df_city[col].isna().all()]
    
    df_clean = df_city.dropna(subset=feature_cols + ['median'])
    
    if len(df_clean) < 200:
        print(f"Insufficient data for {city_name}")
        return None
    
    X = df_clean[feature_cols]
    y = df_clean['median']
    dates = df_clean['Date']
    
    # CV splits
    cv_splits = time_series_cv_split(df_clean, n_splits=5, test_size=30)
    
    # Store results
    all_results = {
        # Statistical
        'ARIMA': {'mae': [], 'rmse': [], 'mape': [], 'family': 'Statistical'},
        'SARIMA': {'mae': [], 'rmse': [], 'mape': [], 'family': 'Statistical'},
        'Exp Smoothing': {'mae': [], 'rmse': [], 'mape': [], 'family': 'Statistical'},
        
        # Machine Learning
        'Random Forest': {'mae': [], 'rmse': [], 'mape': [], 'family': 'Machine Learning'},
        'Gradient Boosting': {'mae': [], 'rmse': [], 'mape': [], 'family': 'Machine Learning'},
        'SVR': {'mae': [], 'rmse': [], 'mape': [], 'family': 'Machine Learning'},
        
        # Deep Learning
        'LSTM': {'mae': [], 'rmse': [], 'mape': [], 'family': 'Deep Learning'},
        'GRU': {'mae': [], 'rmse': [], 'mape': [], 'family': 'Deep Learning'},
        'CNN-LSTM': {'mae': [], 'rmse': [], 'mape': [], 'family': 'Deep Learning'}
    }
    
    print(f"Running 5-Fold Time Series Cross-Validation...")
    
    for fold, (train_idx, test_idx) in enumerate(cv_splits):
        print(f"  Fold {fold+1}/5...", end=' ')
        
        X_train, X_test = X.loc[train_idx], X.loc[test_idx]
        y_train, y_test = y.loc[train_idx], y.loc[test_idx]
        
        # Statistical Models
        pred_arima = train_arima(y_train, y_test)
        pred_sarima = train_sarima(y_train, y_test)
        pred_exp = train_exponential_smoothing(y_train, y_test)
        
        # ML Models
        pred_rf, _ = train_random_forest(X_train, y_train, X_test)
        pred_gb, _ = train_gradient_boosting(X_train, y_train, X_test)
        pred_svr, _ = train_svr(X_train, y_train, X_test)
        
        # DL Models
        pred_lstm, _ = train_lstm(y_train, y_test)
        pred_gru, _ = train_gru(y_train, y_test)
        pred_cnn_lstm, _ = train_cnn_lstm(y_train, y_test)
        
        # Calculate metrics
        predictions = {
            'ARIMA': pred_arima,
            'SARIMA': pred_sarima,
            'Exp Smoothing': pred_exp,
            'Random Forest': pred_rf,
            'Gradient Boosting': pred_gb,
            'SVR': pred_svr,
            'LSTM': pred_lstm,
            'GRU': pred_gru,
            'CNN-LSTM': pred_cnn_lstm
        }
        
        for model_name, pred in predictions.items():
            mae = mean_absolute_error(y_test, pred)
            rmse = np.sqrt(mean_squared_error(y_test, pred))
            mape = mean_absolute_percentage_error(y_test, pred)
            
            all_results[model_name]['mae'].append(mae)
            all_results[model_name]['rmse'].append(rmse)
            all_results[model_name]['mape'].append(mape)
        
        print("✓")
    
    # Calculate averages
    summary = {}
    for model_name in all_results:
        summary[model_name] = {
            'mae': np.mean(all_results[model_name]['mae']),
            'rmse': np.mean(all_results[model_name]['rmse']),
            'mape': np.mean(all_results[model_name]['mape']),
            'mae_std': np.std(all_results[model_name]['mae']),
            'family': all_results[model_name]['family']
        }
    
    # Find best model
    best_model_name = min(summary, key=lambda x: summary[x]['mae'])
    
    print(f"\n{'Model':<20} {'Family':<18} {'MAE':<12} {'RMSE':<12} {'MAPE'}")
    print('-'*80)
    
    # Sort by family for better presentation
    for family in ['Statistical', 'Machine Learning', 'Deep Learning']:
        for model_name, metrics in sorted(summary.items(), key=lambda x: x[1]['mae']):
            if metrics['family'] == family:
                marker = " ⭐" if model_name == best_model_name else ""
                print(f"{model_name:<20} {family:<18} {metrics['mae']:>6.2f} ± {metrics['mae_std']:<3.2f} "
                      f"{metrics['rmse']:>7.2f}     {metrics['mape']:>6.1%}{marker}")
    
    # Retrain best model on full data
    print(f"\nBest Model: {best_model_name} (MAE: {summary[best_model_name]['mae']:.2f})")
    print("Retraining on full dataset...")
    
    if best_model_name == 'Random Forest':
        _, best_model = train_random_forest(X, y, X)
    elif best_model_name == 'Gradient Boosting':
        _, best_model = train_gradient_boosting(X, y, X)
    elif best_model_name == 'SVR':
        _, best_model = train_svr(X, y, X)
    elif best_model_name == 'LSTM':
        _, best_model = train_lstm(y, y)
    elif best_model_name == 'GRU':
        _, best_model = train_gru(y, y)
    elif best_model_name == 'CNN-LSTM':
        _, best_model = train_cnn_lstm(y, y)
    else:
        best_model = None
    
    return {
        'model': best_model,
        'model_name': best_model_name,
        'feature_cols': feature_cols,
        'all_results': summary,
        'X': X,
        'y': y,
        'dates': dates,
        'df_city': df_city
    }

# --- 6. GENERATE FORECASTS ---
def generate_forecast(model_info, forecast_dates):
    """Generate forecasts for January 2026"""
    
    if model_info is None:
        return None
    
    model = model_info['model']
    model_name = model_info['model_name']
    df_city = model_info['df_city']
    
    # For statistical/DL models, use different approach
    if model_name in ['ARIMA', 'SARIMA', 'Exp Smoothing', 'LSTM', 'GRU', 'CNN-LSTM']:
        y_series = df_city['median'].dropna()
        
        if model_name == 'ARIMA':
            fitted = ARIMA(y_series, order=(5, 1, 2)).fit()
            forecast_values = fitted.forecast(steps=31)
        elif model_name == 'SARIMA':
            fitted = SARIMAX(y_series, order=(2, 1, 2), seasonal_order=(1, 1, 1, 7)).fit(disp=False)
            forecast_values = fitted.forecast(steps=31)
        elif model_name == 'Exp Smoothing':
            fitted = ExponentialSmoothing(y_series, seasonal_periods=7, trend='add', seasonal='add').fit()
            forecast_values = fitted.forecast(steps=31)
        else:  # DL models
            lookback = 30
            train_mean, train_std = y_series.mean(), y_series.std()
            y_norm = (y_series - train_mean) / train_std
            
            last_sequence = y_norm.values[-lookback:]
            predictions = []
            
            for _ in range(31):
                X_pred = last_sequence.reshape((1, lookback, 1))
                pred = model.predict(X_pred, verbose=0)[0, 0]
                predictions.append(pred)
                last_sequence = np.append(last_sequence[1:], pred)
            
            forecast_values = np.array(predictions) * train_std + train_mean
        
        residuals = y_series.diff().dropna()
        residual_std = residuals.std()
        
    else:  # ML models
        feature_cols = model_info['feature_cols']
        forecast_df = df_city.copy()
        
        predictions = []
        
        for forecast_date in pd.date_range(forecast_dates[0], forecast_dates[1], freq='D'):
            temp_row = pd.DataFrame({'Date': [forecast_date], 'City': [df_city['City'].iloc[0]]})
            
            temp_row['year'] = forecast_date.year
            temp_row['month'] = forecast_date.month
            temp_row['day'] = forecast_date.day
            temp_row['dayofweek'] = forecast_date.dayofweek
            temp_row['dayofyear'] = forecast_date.dayofyear
            temp_row['quarter'] = forecast_date.quarter
            temp_row['weekofyear'] = forecast_date.isocalendar().week
            temp_row['month_sin'] = np.sin(2 * np.pi * forecast_date.month / 12)
            temp_row['month_cos'] = np.cos(2 * np.pi * forecast_date.month / 12)
            temp_row['day_sin'] = np.sin(2 * np.pi * forecast_date.dayofweek / 7)
            temp_row['day_cos'] = np.cos(2 * np.pi * forecast_date.dayofweek / 7)
            
            for lag in [1, 2, 3, 7, 14, 30]:
                if len(forecast_df) >= lag:
                    temp_row[f'lag_{lag}'] = forecast_df['median'].iloc[-lag]
                else:
                    temp_row[f'lag_{lag}'] = forecast_df['median'].mean()
            
            for window in [7, 14, 30]:
                if len(forecast_df) >= window:
                    temp_row[f'rolling_mean_{window}'] = forecast_df['median'].iloc[-window:].mean()
                    temp_row[f'rolling_std_{window}'] = forecast_df['median'].iloc[-window:].std()
                    temp_row[f'rolling_min_{window}'] = forecast_df['median'].iloc[-window:].min()
                    temp_row[f'rolling_max_{window}'] = forecast_df['median'].iloc[-window:].max()
                else:
                    temp_row[f'rolling_mean_{window}'] = forecast_df['median'].mean()
                    temp_row[f'rolling_std_{window}'] = forecast_df['median'].std()
                    temp_row[f'rolling_min_{window}'] = forecast_df['median'].min()
                    temp_row[f'rolling_max_{window}'] = forecast_df['median'].max()
            
            temp_row['ewm_7'] = forecast_df['median'].iloc[-7:].mean() if len(forecast_df) >= 7 else forecast_df['median'].mean()
            temp_row['ewm_30'] = forecast_df['median'].iloc[-30:].mean() if len(forecast_df) >= 30 else forecast_df['median'].mean()
            
            X_pred = temp_row[feature_cols]
            
            if model_name == 'SVR':
                pred_value = model[0].predict(model[1].transform(X_pred))[0]
            else:
                pred_value = model.predict(X_pred)[0]
            
            predictions.append(pred_value)
            
            temp_row['median'] = pred_value
            forecast_df = pd.concat([forecast_df, temp_row], ignore_index=True)
        
        forecast_values = np.array(predictions)
        residuals = model_info['y'] - model.predict(model_info['X'])
        residual_std = residuals.std()
    
    prediction_dates = pd.date_range(forecast_dates[0], forecast_dates[1], freq='D')
    
    forecast_result = pd.DataFrame({
        'Date': prediction_dates,
        'Prediction': forecast_values,
        'Lower_95': forecast_values - 1.96 * residual_std,
        'Upper_95': forecast_values + 1.96 * residual_std,
        'Lower_80': forecast_values - 1.28 * residual_std,
        'Upper_80': forecast_values + 1.28 * residual_std
    })
    
    return forecast_result

# --- 7. MAIN EXECUTION ---
cities = df['City'].unique()
all_forecasts = {}
all_model_info = {}

for city in cities:
    df_city = create_features(df, city)
    model_info = evaluate_all_models(df_city, city)
    
    if model_info is not None:
        forecast = generate_forecast(model_info, [FORECAST_START, FORECAST_END])
        forecast['City'] = city
        
        all_forecasts[city] = forecast
        all_model_info[city] = model_info

# --- 8. SAVE RESULTS ---
if all_forecasts:
    final_predictions = pd.concat(all_forecasts.values(), ignore_index=True)
    final_predictions.to_csv(os.path.join(PREDICTIONS_DIR, 'pm25_forecast_2026_01.csv'), index=False)
    final_predictions.to_parquet(os.path.join(PREDICTIONS_DIR, 'pm25_forecast_2026_01.parquet'), index=False)
    
    # Save model comparison
    import json
    model_comparison = {}
    for city, info in all_model_info.items():
        model_comparison[city] = {
            'best_model': info['model_name'],
            'all_scores': {k: {'mae': v['mae'], 'rmse': v['rmse'], 'mape': v['mape'], 'family': v['family']} 
                          for k, v in info['all_results'].items()}
        }
    
    with open(os.path.join(OUTPUT_DIR, 'model_comparison.json'), 'w') as f:
        json.dump(model_comparison, f, indent=2)
    
    print("\n" + "="*80)
    print("FORECASTING COMPLETE!")
    print("="*80)
    print(f"✓ Predictions saved: {PREDICTIONS_DIR}")
    print(f"✓ Model comparison saved: {OUTPUT_DIR}/model_comparison.json")
    print("\nBest Models by City:")
    for city, info in all_model_info.items():
        print(f"  {city:15} → {info['model_name']:20} (MAE: {info['all_results'][info['model_name']]['mae']:.2f})")

COMPREHENSIVE PM2.5 FORECASTING PIPELINE
9 Models: 3 Statistical | 3 Machine Learning | 3 Deep Learning

City: Beijing
Running 5-Fold Time Series Cross-Validation...
  Fold 1/5... ✓
  Fold 2/5... ✓
  Fold 3/5... ✓
  Fold 4/5... ✓
  Fold 5/5... ✓

Model                Family             MAE          RMSE         MAPE
--------------------------------------------------------------------------------
ARIMA                Statistical         32.19 ± 12.73   39.24      63.2%
SARIMA               Statistical         32.62 ± 12.75   40.05      61.3%
Exp Smoothing        Statistical         33.39 ± 9.80   41.13      60.6%
Gradient Boosting    Machine Learning    27.02 ± 10.84   33.94      58.7% ⭐
Random Forest        Machine Learning    27.61 ± 10.51   34.23      62.7%
SVR                  Machine Learning    29.35 ± 11.52   35.46      64.6%
GRU                  Deep Learning       31.79 ± 11.78   38.93      69.4%
CNN-LSTM             Deep Learning       32.11 ± 12.96   38.71      64.9%
LSTM    

<h2>96 minutes to train all the models of 6 cities